In [ ]:
import numpy as np
import pandas as pd
import re

import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

pd.set_option('display.max_colwidth',None)

In [ ]:

import chardet

# Detect the encoding of the CSV file
with open('/content/judge-1377884607_tweet_product_company.csv', 'rb') as f:
    encoding = chardet.detect(f.read())['encoding']

print(encoding)

MacRoman


In [ ]:
# Read the CSV file with the detected encoding
data = pd.read_csv('/content/judge-1377884607_tweet_product_company.csv', encoding=encoding)

In [ ]:

data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion


In [ ]:

data.columns

Index(['tweet_text', 'emotion_in_tweet_is_directed_at',
       'is_there_an_emotion_directed_at_a_brand_or_product'],
      dtype='object')

In [ ]:

data.isna().sum()

tweet_text                                               1
emotion_in_tweet_is_directed_at                       5802
is_there_an_emotion_directed_at_a_brand_or_product       0
dtype: int64

In [ ]:

data['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: count, dtype: int64

In [ ]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [ ]:
df = data[['tweet_text','is_there_an_emotion_directed_at_a_brand_or_product']]

In [ ]:
df.head()

,tweet_text,is_there_an_emotion_directed_at_a_brand_or_product
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [ ]:
new_columns = ['tweet','sentiment']
df.columns = new_columns
df.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",Negative emotion
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,Positive emotion
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,Negative emotion
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Positive emotion


In [ ]:
# Define the mapping of sentiment values to numeric values
sentiment_mapping = {
    "No emotion toward brand or product": 0,
    "Positive emotion": 1,
    "Negative emotion": -1,
    "I can't tell": 0
}

# Replace values in the 'sentiment' column using the mapping
df['sentiment'] = df['sentiment'].replace(sentiment_mapping)
df.head()

<ipython-input-14-c4871c69c7dc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace(sentiment_mapping)


,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",-1
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",1
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,1
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,-1
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",1


In [ ]:
df.isna().sum()

tweet        1
sentiment    0
dtype: int64

In [ ]:
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df.isna().sum()

tweet        0
sentiment    0
dtype: int64

In [ ]:
df.head()

,tweet,sentiment
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",-1
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",1
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,1
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,-1
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9092 entries, 0 to 9091
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      9092 non-null   object
 1   sentiment  9092 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 142.2+ KB


In [ ]:
df['cleaned_text'] = df['tweet'].apply(lambda x:gensim.utils.simple_preprocess(x))
df.head()

,tweet,sentiment,cleaned_text
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",-1,"[wesley, have, iphone, after, hrs, tweeting, at, rise_austin, it, was, dead, need, to, upgrade, plugin, stations, at, sxsw]"
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",1,"[jessedee, know, about, fludapp, awesome, ipad, iphone, app, that, you, ll, likely, appreciate, for, its, design, also, they, re, giving, free, ts, at, sxsw]"
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,1,"[swonderlin, can, not, wait, for, ipad, also, they, should, sale, them, down, at, sxsw]"
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,-1,"[sxsw, hope, this, year, festival, isn, as, crashy, as, this, year, iphone, app, sxsw]"
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",1,"[sxtxstate, great, stuff, on, fri, sxsw, marissa, mayer, google, tim, reilly, tech, books, conferences, amp, matt, mullenweg, wordpress]"


In [ ]:

from collections import Counter

words_count = Counter([word for text in df['cleaned_text'] for word in text])
vocab_size = len(words_count)
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 9333


In [ ]:

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:

# Tokenization
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df['cleaned_text'])
X_sequences = tokenizer.texts_to_sequences(df['cleaned_text'])

In [ ]:
# Padding sequences
X_padded = pad_sequences(X_sequences, maxlen=100, padding='post', truncating='post')

In [ ]:
# One-Hot Encoding for sentiment labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df['sentiment'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [ ]:
from keras.layers import Dense, LSTM, Embedding
from keras.models import Sequential

In [ ]:
from keras.layers import Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=5, input_length=100))
model.add(LSTM(50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(60))
model.add(Dense(50, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 5)            46665     
                                                                 
 lstm (LSTM)                 (None, 100, 50)           11200     
                                                                 
 dropout (Dropout)           (None, 100, 50)           0         
                                                                 
 lstm_1 (LSTM)               (None, 60)                26640     
                                                                 
 dense (Dense)               (None, 50)                3050      
                                                                 
 dense_1 (Dense)             (None, 3)                 153       
                                                                 
Total params: 87708 (342.61 KB)
Trainable params: 87708 

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/10
57/57 [==============================] - 23s 285ms/step - loss: 0.8857 - accuracy: 0.6029 - val_loss: 0.8540 - val_accuracy: 0.6003
Epoch 2/10
57/57 [==============================] - 15s 267ms/step - loss: 0.8397 - accuracy: 0.6121 - val_loss: 0.8532 - val_accuracy: 0.6003
Epoch 3/10
57/57 [==============================] - 16s 276ms/step - loss: 0.8389 - accuracy: 0.6121 - val_loss: 0.8535 - val_accuracy: 0.6003
Epoch 4/10
57/57 [==============================] - 15s 270ms/step - loss: 0.8411 - accuracy: 0.6121 - val_loss: 0.8533 - val_accuracy: 0.6003
Epoch 5/10
57/57 [==============================] - 17s 301ms/step - loss: 0.8395 - accuracy: 0.6121 - val_loss: 0.8529 - val_accuracy: 0.6003
Epoch 6/10
57/57 [==============================] - 16s 274ms/step - loss: 0.8383 - accuracy: 0.6121 - val_loss: 0.8532 - val_accuracy: 0.6003
Epoch 7/10
57/57 [==============================] - 16s 273ms/step - loss: 0.8393 - accuracy: 0.6121 - val_loss: 0.8535 - val_accuracy: 0.6003